In [20]:
import pandas as pd
from fuzzywuzzy import process

In [21]:
data=pd.read_csv('./data/world_university_rank.csv').dropna()
data.columns = ['Rank','University name', 'Country', 'Number of students', 'Number of students per staff', 'Percentage of international students', 'Gender ratio']
print(len(data))
data.head()

2152


,Rank,University name,Country,Number of students,Number of students per staff,Percentage of international students,Gender ratio
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52
1,2,Harvard University,United States,"21,887",9.6,25%,50 : 50
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53
3,3,Stanford University,United States,"16,164",7.1,24%,46 : 54
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60


In [22]:
mapping=pd.read_csv('./data/countryContinent.csv', encoding = "ISO-8859-1").dropna()
print(len(mapping))
mapping.head()

239


,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [4]:
new_data = data.copy()

In [6]:
choices = mapping['country'].values.tolist()
data['Close Country'] = data['Country'].apply(lambda x: process.extractOne(x, choices)[0])

# merge
data = data.merge(mapping[['country','sub_region']], left_on='Close Country', right_on='country')
data = data.drop(['country','Close Country'], axis=1).sort_values(by=['Rank'], key=lambda x: x.astype(str).str.zfill(3))
data.rename(columns={'sub_region':'SubRegion'}, inplace=True)
data.head()

,Rank,University name,Country,Number of students,Number of students per staff,Percentage of international students,Gender ratio,SubRegion
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52,Northern Europe
163,2,Harvard University,United States,"21,887",9.6,25%,50 : 50,Northern America
1,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53,Northern Europe
164,3,Stanford University,United States,"16,164",7.1,24%,46 : 54,Northern America
165,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60,Northern America


In [15]:
#merge again to preserve intial order
new_data=new_data.merge(data[['University name','SubRegion']], on='University name')
#no data loss
print(len(new_data))

2152


In [17]:
new_data.head()

,Rank,University name,Country,Number of students,Number of students per staff,Percentage of international students,Gender ratio,SubRegion
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52,Northern Europe
1,2,Harvard University,United States,"21,887",9.6,25%,50 : 50,Northern America
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53,Northern Europe
3,3,Stanford University,United States,"16,164",7.1,24%,46 : 54,Northern America
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60,Northern America


In [19]:
new_data.SubRegion.unique()

array(['Northern Europe', 'Northern America', 'Western Europe',
       'South-Eastern Asia', 'Eastern Asia', 'Australia and New Zealand',
       'Western Asia', 'Southern Africa', 'Southern Europe',
       'South America', 'Eastern Europe', 'Southern Asia',
       'Western Africa', 'Northern Africa', 'Eastern Africa', 'Caribbean',
       'Central America', 'Central Asia', 'Melanesia'], dtype=object)

In [16]:
new_data.to_json('world_university_rank.json',orient='records')